# 20. Project: 모델 바꿔보기

**simplebaseline 모델로 변경**해 봅시다

지금까지 우리는 **StackedHourglass Network** 기반으로 학습을 진행해 왔습니다.

그러나 지난시간에 살펴본 것처럼 **Simplebaseline** 모델은 이보다 훨씬 간단한 모델 구조에도 불구하고 더욱 좋은 성능을 보여주었던 바 있습니다.

실제로도 그런 성능을 얻을 수 있을지 확인해 보겠습니다.

## 루브릭

|평가문항|	상세기준|
|:-------|:-------|
|1. tfrecord를 활용한 데이터셋 구성과 전처리를 통해 프로젝트 베이스라인 구성을 확인하였다.|MPII 데이터셋을 기반으로 1epoch에 30분 이내에 학습가능한 베이스라인을 구축하였다.|
|2. simplebaseline 모델을 정상적으로 구현하였다.|simplebaseline 모델을 구현하여 실습코드의 모델을 대체하여 정상적으로 학습이 진행되었다.|
|3. Hourglass 모델과 simplebaseline 모델을 비교분석한 결과를 체계적으로 정리하였다.|두 모델의 pose estimation 테스트결과 이미지 및 학습진행상황 등을 체계적으로 비교분석하였다.|

In [1]:
import tensorflow as tf

print(tf.__version__)
# 2.4.1

tf.test.is_gpu_available(
    cuda_only=False,
    min_cuda_compute_capability=None
)
# True

2.4.1
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

# STEP 1 : simplebaseline 모델 완성하기
**simplebaseline.py** 파일 내용을 완성합니다.


* os.getenv('HOME')+'/aiffel/mpii/' 폴더 밑에 `simplebaseline.py`파일로 저장됨

## simplebaseline.py

In [2]:
import tensorflow as tf


def _make_deconv_layer(num_deconv_layers):
    seq_model = tf.keras.models.Sequential()

    for i in range(num_deconv_layers):
        seq_model.add(tf.keras.layers.Conv2DTranspose(256, kernel_size=(4,4), strides=(2,2), padding='same'))
        seq_model.add(tf.keras.layers.BatchNormalization())
        seq_model.add(tf.keras.layers.ReLU())

    return seq_model

resnet = tf.keras.applications.resnet.ResNet50(include_top=False, weights='imagenet')
upconv = _make_deconv_layer(3)
final_layer = tf.keras.layers.Conv2D(17, kernel_size=(1,1), padding='same')


def Simplebaseline(input_shape=(256, 256, 3)):
    inputs = tf.keras.Input(shape=input_shape)
    x = resnet(inputs)
    x = upconv(x)
    out = final_layer(x)

    model = tf.keras.Model(inputs, out, name='simple_baseline')
    
    return model


base_model = Simplebaseline()
base_model.summary()

Model: "simple_baseline"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, None, None, 2048)  23587712  
_________________________________________________________________
sequential (Sequential)      (None, 64, 64, 256)       10489600  
_________________________________________________________________
conv2d (Conv2D)              (None, 64, 64, 17)        4369      
Total params: 34,081,681
Trainable params: 34,027,025
Non-trainable params: 54,656
_________________________________________________________________


# STEP 2 : simplebaseline 모델로 변경하여 훈련하기
**train.py 218라인의 모델 선언 부분을 simplebaseline 모델로 변경한 후 다시 학습**을 진행합니다.

* model = StackedHourglassNetwork(IMAGE_SHAPE, 4, 1, num_heatmap) 부분을  
  model = Simplebaseline(IMAGE_SHAPE) 으로 수정
  
  
* tfrecord 생성
  - cd ~/aiffel/mpii && python tfrecords_mpii.py

# STEP 3 : 두 모델의 비교
실습에서 다룬 **StackedHourglass Network와 Simplebaseline 모델을 둘 다 동일한 Epoch 수만큼 학습하여 그 결과를 비교**해 봅니다.

* Pose Estimation 결과 시각화 (정성적 비교)
* 학습 진행경과 (loss 감소현황)

가급적 두 모델 공히 최소 3epoch이상, (5epoch 이상 권장)을 학습하기 바랍니다.

# 회고

* tfrecord 생성 시켰는데 파일 용량이 모두 0 byte로 되어 있으며 내용이 비어 있다.